In [ ]:
!pip install ultralytics==8.2.103
!pip install roboflow
!pip install inference-sdk
!pip install streamlit
!pip install opencv-python-headless pillow
!npm install -g localtunnel
!npm i --package-lock-only
!npm init -y
!npm audit fix
!npm install express
!npm audit


changed 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
npm ERR! code ENOENT
npm ERR! syscall open
npm ERR! path /content/package.json
npm ERR! errno -2
npm ERR! enoent ENOENT: no such file or directory, open '/content/package.json'
npm ERR! enoent This is related to npm not being able to find a file.
npm ERR! enoent 

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-11-22T09_02_22_430Z-debug-0.log
Wrote to /content/package.json:

{
  "name": "content",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}



up to date, audited 1 package in 139ms

found 0 vulnerabilities

added 65 packages, and audite

In [ ]:
%%writefile WildAnimalAlertUI.py
import streamlit as st
import cv2
from email.message import EmailMessage
import smtplib
from roboflow import Roboflow
from inference_sdk import InferenceHTTPClient, InferenceConfiguration
import tempfile
import os
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

# Roboflow and Email Configuration
rf = Roboflow(api_key="NJvbl1HBuiiRBzqivAcU")
project = rf.workspace("dhruvs-workspace").project("animalintusiondetectionsystem")
version = project.version(3)
custom_configuration = InferenceConfiguration(confidence_threshold=0.4)
CLIENT = InferenceHTTPClient(api_url="https://detect.roboflow.com", api_key="NJvbl1HBuiiRBzqivAcU")

sender_email = "wildanimalsalert@gmail.com"
sender_password = "kvlpbtvuelasjbgs"

def emailalerts(subject, body, to):
    try:
        msg = EmailMessage()
        msg.set_content(body)
        msg['Subject'] = subject
        msg['To'] = to
        msg['From'] = sender_email

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
        server.quit()
        st.success(f"Email sent successfully to {to}")
    except Exception as e:
        st.error(f"Failed to send email. Error: {e}")

def display_consecutive_frames(frames, predictions_list):
    """
    Display the detected frames using matplotlib.
    """
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    axes = axes.flatten()

    for idx, (frame, predictions) in enumerate(zip(frames, predictions_list)):
        ax = axes[idx]
        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        for pred in predictions:
            x, y, w, h = pred['x'], pred['y'], pred['width'], pred['height']
            label = pred['class']
            conf = pred['confidence']
            rect = Rectangle((x - w / 2, y - h / 2), w, h, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            ax.text(x - w / 2, y - h / 2 - 10, f"{label} ({conf:.2f})", color='red', fontsize=10, backgroundcolor='white')
        ax.axis('off')
    plt.tight_layout()
    st.pyplot(fig)

def process_video(video_path, recipient_email):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    consecutive_detections = 0
    required_consecutive_frames = 12  # Threshold for sending an alert
    detection_frames = []  # Store frames with detections
    detection_predictions = []  # Store predictions for those frames
    email_sent = False  # Flag to ensure the email is sent only once

    while cap.isOpened() and not email_sent:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        temp_filename = f"frame_{frame_count}.jpg"
        cv2.imwrite(temp_filename, frame)

        # Run inference
        result = CLIENT.infer(temp_filename, model_id="animalintusiondetectionsystem/3")
        os.remove(temp_filename)  # Clean up temporary file
        predictions = result['predictions']

        if predictions:
            consecutive_detections += 1
            detection_frames.append(frame)
            detection_predictions.append(predictions)
        else:
            consecutive_detections = 0
            detection_frames.clear()
            detection_predictions.clear()

        # Check if threshold is met
        if consecutive_detections >= required_consecutive_frames and not email_sent:
            body = [f"{pred['class']}" for pred in predictions]
            detected_animals = ' and '.join(body)
            subject = f"{len(predictions)} Animal(s) detected"
            email_body = f"{detected_animals} detected near your area. Please report to the authorities."
            emailalerts(subject=subject, body=email_body, to=recipient_email)

            # Display detected frames
            display_consecutive_frames(detection_frames, detection_predictions)

            # Set the email_sent flag to True
            email_sent = True

    cap.release()
    if not email_sent:
        st.info("No significant detections in the video.")

# Streamlit UI
st.title("Wild Animal Intrusion Detection System")
st.write("Upload a photo or video and provide your email to detect animals and get alerts.")

# Upload file
uploaded_file = st.file_uploader("Upload Video or Image", type=["mp4", "mov", "jpg", "png"])
recipient_email = st.text_input("Enter your email ID:")

if st.button("Process"):
    if uploaded_file and recipient_email:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(uploaded_file.read())
            temp_file_path = temp_file.name

        if uploaded_file.type.startswith("video"):
            st.info("Processing video...")
            process_video(temp_file_path, recipient_email)
        elif uploaded_file.type.startswith("image"):
            st.info("Processing image...")
            # Run inference for the image
            result = CLIENT.infer(temp_file_path, model_id="animalintusiondetectionsystem/3")
            predictions = result['predictions']
            frame = cv2.imread(temp_file_path)

            if predictions:
                display_consecutive_frames([frame], [predictions])
                body = [f"{pred['class']}" for pred in predictions]
                detected_animals = ' and '.join(body)
                subject = f"{len(predictions)} Animal(s) detected"
                email_body = f"{detected_animals} detected in the uploaded image. Please report to the authorities."
                emailalerts(subject=subject, body=email_body, to=recipient_email)
            else:
                st.info("No significant detections in the image.")
        else:
            st.error("Unsupported file type.")

        # Clean up temporary file
        os.remove(temp_file_path)
    else:
        st.error("Please upload a file and provide an email address.")

Writing WildAnimalAlertUI.py


In [ ]:
!curl ipv4.icanhazip.com

34.169.1.172


In [ ]:
import subprocess
process = subprocess.Popen(['streamlit', 'run', 'WildAnimalAlertUI.py'])

# Wait for the app to start
import time
time.sleep(5)  # Wait a few seconds to let the app start

# Verify Streamlit app locally
!curl http://localhost:8501

# Start LocalTunnel to expose the Streamlit app
!lt --port 8501

<!doctype html><html lang="en"><head><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><link rel="shortcut icon" href="./favicon.png"/><link rel="preload" href="./static/media/SourceSansPro-Regular.0d69e5ff5e92ac64a0c9.woff2" as="font" type="font/woff2" crossorigin><link rel="preload" href="./static/media/SourceSansPro-SemiBold.abed79cd0df1827e18cf.woff2" as="font" type="font/woff2" crossorigin><link rel="preload" href="./static/media/SourceSansPro-Bold.118dea98980e20a81ced.woff2" as="font" type="font/woff2" crossorigin><title>Streamlit</title><script>window.prerenderReady=!1</script><script defer="defer" src="./static/js/main.75ac1cb6.js"></script><link href="./static/css/main.23bdda6f.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div></body></html>your url is: https://tidy-experts-grin.loca.lt
^C
